In [1]:
import numpy as np
import time
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import json
import requests
import datetime
from datetime import timedelta
import threading
import sys
from flask import Flask, request, jsonify
import socketio
import talib
import hmac
import hashlib
from pyts.image import GramianAngularField
from tensorflow.nn import softmax
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Global variables initialization
current_btc_price = None
cnn_model = None
lstm_model = None
order_status = None
app = Flask(__name__)
upper_quantile = None
lower_quantile = None
lstm_mae = 2000
models_loaded = False
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
live_data = []
last_processed_minute = None
current_status = ""
lookback = 10
window_size = 10
order_status = "filled"
paper_trade = None
last_price = None
current_prediction_status = ""
last_status_length = 0  # Track length of last status message

# Thread lock for trade operations
trade_lock = threading.Lock()

# Custom activation function
def swish(x):
    return tf.nn.swish(x)

# Register the custom activation
tf.keras.utils.get_custom_objects().update({'swish': tf.keras.activations.get(swish)})

def fetch_balance():
    key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
    secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
    
    secret_bytes = bytes(secret, encoding='utf-8')
    timeStamp = int(round(time.time() * 1000))
    
    body = {"timestamp": timeStamp}
    json_body = json.dumps(body, separators=(',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    url = "https://api.coindcx.com/exchange/v1/users/balances"
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }
    
    response = requests.post(url, data=json_body, headers=headers)
    data = response.json()
    btc_balance = next((item['balance'] for item in data if item['currency'] == 'INR'), None)
    return btc_balance

def place_order(order_type_buy_sell, price, shares):
    global current_btc_price, order_status
    if 1==1:
        if order_status == 'filled':
            key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
            secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
            
            secret_bytes = bytes(secret, encoding='utf-8')
            timeStamp = int(round(time.time() * 1000))
            
            # if order_type_buy_sell.lower() == "buy":
            #     price_per_unit = current_btc_price * 1.001
            # else:
            #     price_per_unit = current_btc_price * 0.999
            if order_type_buy_sell.lower() == "buy":
                price_per_unit = price
            else:
                price_per_unit = price
            
            body = {
                "side": order_type_buy_sell,
                "order_type": "limit_order",
                "market": "BTCINR",
                "total_quantity": 0.0002,
                "timestamp": timeStamp,
                "price_per_unit": round(current_btc_price,2)
            }
            
            json_body = json.dumps(body, separators=(',', ':'))
            signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
            
            url = "https://api.coindcx.com/exchange/v1/orders/create"
            headers = {
                'Content-Type': 'application/json',
                'X-AUTH-APIKEY': key,
                'X-AUTH-SIGNATURE': signature
            }
            
            print(f"\nPlacing {order_type_buy_sell} order: {shares} BTC at {price_per_unit:.2f} (Market: {current_btc_price:.2f})")
            
            response = requests.post(url, data=json_body, headers=headers)
            if response.status_code == 200:
                try:
                    order_response = response.json()
                    print("Order response:", order_response)
                    # order_status = 'open'
                    # print(f"Order response: {order_response}")
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON: {e}")
                    print(f"Response text: {response.text}")
            else:
                print(f"Request failed with status code {response.status_code}")
                print(f"Response text: {response.text}")
        else:
            print("Order already pending")
        return None

def update_status_display(status):
    """Helper function to update status display on same line"""
    global last_status_length
    # Clear the previous status by overwriting with spaces
    sys.stdout.write('\r' + ' ' * last_status_length)
    # Write the new status
    sys.stdout.write('\r' + status)
    sys.stdout.flush()
    last_status_length = len(status)

def clean_data(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df.fillna(0, inplace=True)
    return df

def load_models(models='both'):
    global lstm_model, cnn_model, upper_quantile, lower_quantile, models_loaded
    try:
        custom_objects = {'swish': swish}
        
        if models == "both" or models == "LSTM":
            lstm_model = tf.keras.models.load_model(
                "lstm_model_coindcx_optimized.h5", 
                compile=False,
                custom_objects=custom_objects
            )
            print("LSTM Model loaded successfully.")
        
        if models == "both" or models == "CNN":
            cnn_model = tf.keras.models.load_model(
                "cnn_model_coindcx_optimized.h5", 
                compile=False,
                custom_objects=custom_objects
            )
            print("CNN Model loaded successfully.")
        
        if models == "both" or models == "QUANTILES":
            upper_quantile = np.load("upper_quantile_coindcx.npy")
            lower_quantile = np.load("lower_quantile_coindcx.npy")
            print("Quantiles loaded successfully.")
        
        if lstm_model is not None and cnn_model is not None:
            models_loaded = True
            print("All models are ready for prediction.")
        else:
            print("Warning: Some models failed to load")
            
    except Exception as e:
        print(f"Error loading models: {e}")
        models_loaded = False

def fetch_and_preprocess_data():
    end_time = int(datetime.datetime.now().timestamp() * 1000)
    start_time = int((datetime.datetime.now() - timedelta(days=5)).timestamp() * 1000)

    url = f"https://public.coindcx.com/market_data/candles?pair=I-ETH_INR&interval=1m&startTime={start_time}&endTime={end_time}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if not isinstance(data, list) or len(data) == 0:
            print("Error: API returned invalid or empty data.")
            return None, None, None

        btc_data = pd.DataFrame(data)
        btc_data = btc_data.rename(columns={
            'open': 'Open', 
            'high': 'High', 
            'low': 'Low', 
            'volume': 'Volume', 
            'close': 'Close'
        })

        btc_data['time'] = pd.to_datetime(btc_data['time'], unit='ms')
        btc_data[['Open', 'High', 'Low', 'Close', 'Volume']] = btc_data[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
        btc_data = clean_data(btc_data)

        try:
            btc_data['rsi'] = talib.RSI(btc_data['Close'], timeperiod=14)
            btc_data['macd'], btc_data['macd_signal'], btc_data['macd_hist'] = talib.MACD(
                btc_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9
            )
            btc_data['upperband'], btc_data['middleband'], btc_data['lowerband'] = talib.BBANDS(
                btc_data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
            )
            btc_data['ema_50'] = talib.EMA(btc_data['Close'], timeperiod=50)
            btc_data['ema_200'] = talib.EMA(btc_data['Close'], timeperiod=200)
            
            btc_data = clean_data(btc_data)
            btc_data['Next_Close'] = btc_data['Close'].shift(-10)
            btc_data.dropna(inplace=True)
            
            feature_cols = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 
                          'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
            
            btc_data[feature_cols] = scaler_x.fit_transform(btc_data[feature_cols])
            btc_data['Next_Close'] = scaler_y.fit_transform(btc_data[['Next_Close']])
            
            print(f"Data loaded successfully. Rows: {len(btc_data)}")
            return btc_data, scaler_x, scaler_y
            
        except Exception as e:
            print(f"Error calculating indicators: {e}")
            return None, None, None
            
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None, None, None

def preprocess_data(data):
    try:
        if data is None or not isinstance(data, pd.DataFrame):
            print("Error: Invalid input data")
            return None

        if 'Close' not in data.columns:
            print("Error: 'Close' column missing")
            return None

        data['rsi'] = talib.RSI(data['Close'], timeperiod=14)
        data['macd'], data['macd_signal'], data['macd_hist'] = talib.MACD(
            data['Close'], fastperiod=12, slowperiod=26, signalperiod=9
        )
        data['upperband'], data['middleband'], data['lowerband'] = talib.BBANDS(
            data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
        )
        data['ema_50'] = talib.EMA(data['Close'], timeperiod=50)
        data['ema_200'] = talib.EMA(data['Close'], timeperiod=200)
        
        data = clean_data(data)
        return data
        
    except Exception as e:
        print(f"Error in preprocess_data: {e}")
        return None

def calculate_dynamic_factors(close_diff):
    # if close_diff >= 100000:
    #     return 0.1, 0.1
    # elif close_diff >= 50000:
    #     return 0.2, 0.2
    # elif close_diff >= 10000:
    #     return 0.3, 0.3
    # elif close_diff >= 5000:
    #     return 0.4, 0.4
    # else:
    return 0.1, 0.5

# class PaperTrade:
#     def __init__(self, balance=12000):
#         self.balance = balance
#         self.position = None  # 'long', 'short', or None
#         self.entry_price = None
#         self.stop_loss = None
#         self.target = None
#         self.shares = 0
#         self.investment_per_trade = 22  # INR amount per trade
#         self.taker_fee = 0.0005  # 0.05%
#         self.maker_fee = 0.0002  # 0.02%
#         self.trade_history = []  # To track all trades

#     def calculate_dynamic_factors(self, price_change):
#         """Calculate dynamic stop-loss and take-profit percentages"""
#         volatility_factor = min(max(abs(price_change) / 1000, 0.005), 0.02)  # Between 0.5% and 2%
#         target_pct = volatility_factor * 2  # 2x volatility for target
#         stop_loss_pct = volatility_factor
#         return target_pct, stop_loss_pct

#     def execute_trade(self, signal, price, predicted_close_diff):
#         global order_status, current_btc_price
        
#         if order_status != 'filled':
#             return

#         target_pct, stop_loss_pct = self.calculate_dynamic_factors(predicted_close_diff)
#         trade_log = ""
#         fee = 0
#         trade_amount = self.investment_per_trade
        
#         # LONG ENTRY
#         if signal == 1 and self.position is None:
#             if self.balance >= trade_amount:
#                 self.position = "long"
#                 self.entry_price = price
#                 self.stop_loss = price * (1 - stop_loss_pct)
#                 self.target = price * (1 + target_pct)
#                 self.shares = trade_amount / price  # Calculate BTC quantity
                
#                 fee = self.entry_price * self.shares * self.maker_fee
#                 self.balance -= (trade_amount + fee)
                
#                 trade_log = (f"🚀📈 LONG ENTRY: {price:.2f} | "
#                            f"Target: {self.target:.2f} | "
#                            f"SL: {self.stop_loss:.2f} | "
#                            f"Qty: {self.shares:.6f} BTC | "
#                            f"Fee: {fee:.4f} INR | "
#                            f"Bal: {self.balance:.2f} INR")
                
#                 # Place real order (uncomment when ready)
#                 # place_order("buy", price, self.shares)
                
#         # SHORT ENTRY
#         elif signal == 2 and self.position is None:
#             if self.balance >= trade_amount:
#                 self.position = "short"
#                 self.entry_price = price
#                 self.stop_loss = price * (1 + stop_loss_pct)
#                 self.target = price * (1 - target_pct)
#                 self.shares = trade_amount / price  # Calculate BTC quantity
                
#                 fee = self.entry_price * self.shares * self.maker_fee
#                 self.balance -= fee  # Only fee for short entry
                
#                 trade_log = (f"📉🔻 SHORT ENTRY: {price:.2f} | "
#                            f"Target: {self.target:.2f} | "
#                            f"SL: {self.stop_loss:.2f} | "
#                            f"Qty: {self.shares:.6f} BTC | "
#                            f"Fee: {fee:.4f} INR | "
#                            f"Bal: {self.balance:.2f} INR")
                
#                 # Place real order (uncomment when ready)
#                 # place_order("sell", price, self.shares)
                
#         # LONG EXIT
#         elif self.position == "long" and (price >= self.target or price <= self.stop_loss):
#             exit_value = price * self.shares
#             pnl = (price - self.entry_price) * self.shares
#             fee = exit_value * self.maker_fee
#             self.balance += (exit_value - fee)
            
#             trade_log = (f"✅📈 LONG EXIT: {price:.2f} | "
#                        f"PnL: {pnl:.4f} INR | "
#                        f"Fee: {fee:.4f} INR | "
#                        f"Bal: {self.balance:.2f} INR")
            
#             # Reset position
#             self.position = None
#             self.shares = 0
            
#             # Place real order (uncomment when ready)
#             # place_order("sell", price, self.shares)
            
#         # SHORT EXIT
#         elif self.position == "short" and (price <= self.target or price >= self.stop_loss):
#             exit_value = price * self.shares
#             pnl = (self.entry_price - price) * self.shares
#             fee = exit_value * self.maker_fee
#             self.balance += (self.investment_per_trade + pnl - fee)
            
#             trade_log = (f"✅📉 SHORT EXIT: {price:.2f} | "
#                        f"PnL: {pnl:.4f} INR | "
#                        f"Fee: {fee:.4f} INR | "
#                        f"Bal: {self.balance:.2f} INR")
            
#             # Reset position
#             self.position = None
#             self.shares = 0
            
#             # Place real order (uncomment when ready)
#             # place_order("buy", price, self.shares)

#         # Log the trade if executed
#         if trade_log:
#             print(f"\n{trade_log}")
#             self.trade_history.append({
#                 'timestamp': datetime.datetime.now(),
#                 'type': self.position,
#                 'entry_price': self.entry_price,
#                 'exit_price': price,
#                 'shares': self.shares,
#                 'pnl': pnl if 'pnl' in locals() else 0,
#                 'fee': fee,
#                 'balance': self.balance
#             })
            
#             with open("trading_log_btc_inr.txt", "a") as log_file:
#                 log_file.write(trade_log + "\n")

#     def get_portfolio_summary(self):
#         """Return current portfolio status"""
#         return {
#             'balance': self.balance,
#             'position': self.position,
#             'entry_price': self.entry_price,
#             'current_value': self.shares * current_btc_price if self.shares > 0 else 0,
#             'unrealized_pnl': (current_btc_price - self.entry_price) * self.shares if self.position == 'long' else 
#                              (self.entry_price - current_btc_price) * self.shares if self.position == 'short' else 0,
#             'num_trades': len(self.trade_history)
#         }


class PaperTrade:
    def __init__(self, balance=12000):
        self.balance = balance
        self.position = None
        self.entry_price = None
        self.stop_loss = None
        self.target = None
        self.shares = 0
        self.investment_per_trade = 22
        self.taker_fee = 0.0005
        self.maker_fee = 0.0002

    def execute_trade(self, signal, price, predicted_close_diff):
        global current_status, current_prediction_status, order_status, current_btc_price
        # if signal == 1:
        #     print(signal, price, predicted_close_diff)
        if 1==1:
            if order_status != 'filled':
                return

            target_pct, stop_loss_pct = calculate_dynamic_factors(abs(predicted_close_diff))
            trade_log = ""
            fee = 0
            
            if signal == 1 and self.position is None:
                if self.balance >= self.investment_per_trade:
                    self.position = "long"
                    self.entry_price = price
                    self.stop_loss = price - abs(predicted_close_diff) * stop_loss_pct
                    self.target = price + abs(predicted_close_diff) * target_pct
                    self.shares = 0.0002
                    
                    fee = self.entry_price * self.shares * self.maker_fee
                    self.balance -= fee  # Only fee for short entry
                    # place_order("buy", price, self.shares)
                    # self.balance = fetch_balance()
                    trade_log = (f"🚀📈 LONG ENTRY: {price:.2f} | "
                               f"Target: {self.target:.2f} | "
                               f"SL: {self.stop_loss:.2f} | "
                               f"Qty: {self.shares} | "
                               f"Fee: {fee:.2f} (0.05%) | "
                               f"Bal: {self.balance:.2f}")

            elif signal == 2 and self.position is None:
                if self.balance >= self.investment_per_trade:
                    self.position = "short"
                    self.entry_price = price
                    self.stop_loss = price + abs(predicted_close_diff) * stop_loss_pct
                    self.target = price - abs(predicted_close_diff) * target_pct
                    self.shares = 0.0001
                    
                    fee = self.entry_price * self.shares * self.maker_fee
                    self.balance -= fee
                    
                    trade_log = (f"📉🔻 SHORT ENTRY: {price:.2f} | "
                               f"Target: {self.target:.2f} | "
                               f"SL: {self.stop_loss:.2f} | "
                               f"Qty: {self.shares} | "
                               f"Fee: {fee:.2f} (0.05%) | "
                               f"Bal: {self.balance:.2f}")
                
            elif self.position == "long" and (price >= self.target or price <= self.stop_loss):
                exit_value = price * self.shares
                pnl = (price - self.entry_price) * self.shares
                fee = exit_value * self.maker_fee
                self.balance += (pnl - fee)
                # place_order("sell", price, self.shares)
                # self.balance = fetch_balance()
                trade_log = (f"✅📈 LONG EXIT: {price:.2f} | "
                            f"PnL: {pnl:.2f} | "
                            f"Fee: {fee:.2f} (0.02%) | "
                            f"Bal: {self.balance:.2f}")

            elif self.position == "short" and (price <= self.target or price >= self.stop_loss):
                pnl = (self.entry_price - price) * self.shares
                fee = price * self.shares * self.maker_fee
                
                self.balance += (pnl - fee)
                self.position = None
                
                trade_log = (f"✅📉 SHORT EXIT: {price:.2f} | "
                            f"PnL: {pnl:.2f} | "
                            f"Fee: {fee:.2f} (0.02%) | "
                            f"Bal: {self.balance:.2f}")

            if trade_log:
                print(f"\n{trade_log}")
                with open("15_may_eth_inr__2", "a") as log_file:
                    log_file.write(trade_log + "\n")

def predict(list_of_close):
    global current_status
    
    try:
        if not models_loaded or lstm_model is None or cnn_model is None:
            current_status = "Models not ready"
            return
            
        df = pd.DataFrame(list_of_close, columns=["Close"])
        df = preprocess_data(df)
        
        if df is None:
            current_status = "Preprocessing failed"
            return
            
        features = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 
                   'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
        
        missing_features = [f for f in features if f not in df.columns]
        if missing_features:
            current_status = f"Missing features: {missing_features}"
            return
            
        scaled_data = scaler_x.transform(df[features])
        
        if np.isnan(scaled_data).any() or np.isinf(scaled_data).any():
            current_status = "Scaled data contains NaN/inf"
            return
            
        if scaled_data.shape[0] < lookback:
            return
    
        lstm_X = np.array([scaled_data[-lookback:]]).reshape((1, lookback, len(features)))
        predicted_next_close_scaled = lstm_model.predict(lstm_X, verbose=0)[0][0]
        predicted_next_close = scaler_y.inverse_transform([[predicted_next_close_scaled]])[0][0]
        
        gaf = GramianAngularField(method="summation")
        gaf_image = gaf.fit_transform(scaled_data[-window_size:].T)
        cnn_X = np.expand_dims(gaf_image, axis=0)
        cnn_X = np.transpose(cnn_X, (0, 2, 3, 1))
        
        cnn_pred = cnn_model.predict(cnn_X, verbose=0)
        class_probabilities = softmax(cnn_pred).numpy()[0]
        predicted_class = np.argmax(cnn_pred, axis=1)[0]
        predicted_class_prob = class_probabilities[predicted_class]
    
        close_diff = predicted_next_close - list_of_close[-1]
        signal = 0
        
        if close_diff > 0 and predicted_class == 1:
            signal = 1
        # if close_diff > -lstm_mae/2 and predicted_class == 2:
        #     signal = 2
        paper_trade.execute_trade(signal, list_of_close[-1], close_diff)
        
        current_status = (
            f"Price: {list_of_close[-1]:.2f}, Balance: {paper_trade.balance:.2f}, "
            f"Position: {paper_trade.position if paper_trade.position else 'None'}, "
            f"Shares: {paper_trade.shares}, Next_Close: {predicted_next_close:.2f}, "
            f"Close_Diff: {close_diff:.2f}, Signal: {signal}, "
            f"Order Status: {order_status}, LSTM: {close_diff}, "
            f"CNN: {predicted_class}, CNN Prob.: {predicted_class_prob}"
        )
        
        update_status_display(current_status)
    except Exception as e:
        current_status = f"Error in predict: {e}"
        update_status_display(current_status)

def start_websocket():
    socketEndpoint = 'wss://stream.coindcx.com'
    sio = socketio.Client()

    @sio.event
    def connect():
        print("Connected to CoinDCX WebSocket!")
        sio.emit('join', {'channelName': "I-ETH_INR@prices"})

    @sio.on('price-change')
    def on_message(message):
        global last_processed_minute, live_data, last_price, current_btc_price
        
        try:
            btc_inr_price = float(json.loads(message['data'])['p'])
            last_price = btc_inr_price
            current_btc_price = btc_inr_price
            
            current_time = datetime.datetime.now()
            current_minute = current_time.minute
            
            if last_processed_minute is None:
                last_processed_minute = current_minute

            if current_minute != last_processed_minute:
                last_processed_minute = current_minute

                if len(live_data) >= lookback:
                    live_data.pop(0)

                live_data.append(btc_inr_price)
                data_to_pass = live_data.copy()
            else:
                data_to_pass = live_data[-(lookback-1):] + [btc_inr_price]
                
            predict(data_to_pass)
            
        except Exception as e:
            current_status = f"Error in websocket: {e}"
            update_status_display(current_status)

    sio.connect(socketEndpoint, transports=['websocket'])
    sio.wait()

def order_and_balance_update():
    global order_status
    
    socketEndpoint = 'wss://stream.coindcx.com'
    
    key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
    secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
    
    secret_bytes = bytes(secret, encoding='utf-8')
    channelName = "coindcx"
    body = {"channel": channelName}
    json_body = json.dumps(body, separators=(',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    sio2 = socketio.Client()

    @sio2.event
    def connect():
        print("Connected to order channel!")
        sio2.emit('join', {'channelName': channelName, 'authSignature': signature, 'apiKey': key})
    
    @sio2.on('order-update')
    def on_message1(response):
        global order_status
        if isinstance(response, dict) and 'data' in response:
            try:
                order_data = json.loads(response['data'])
                if isinstance(order_data, list) and len(order_data) > 0:
                    status = order_data[0].get('status', 'Unknown')
                    order_status = status
            except json.JSONDecodeError:
                pass

    sio2.connect(socketEndpoint, transports=['websocket'])
    sio2.wait()

@app.route('/update', methods=['POST'])
def update():
    global lstm_mae
    data = request.get_json()
    if 'Updated' in data:
        load_models(models=data['Updated'])
        return jsonify({'message': 'Update received successfully'}), 200
    if 'LSTM' in data:
        lstm_mae = int(data['LSTM'])
        return jsonify({'message': 'LSTM mae received successfully'}), 200
    return jsonify({'error': 'Missing parameter'}), 400

if __name__ == '__main__':
    load_models(models='both')
    btc_data, scaler_x, scaler_y = fetch_and_preprocess_data()

    if btc_data is None or scaler_x is None or scaler_y is None:
        print("Failed to initialize data. Exiting...")
        sys.exit(1)

    live_data = btc_data['Close'].tolist()[-lookback:]
    paper_trade = PaperTrade(balance=10000)

    threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)).start()
    # threading.Thread(target=order_and_balance_update).start()
    threading.Thread(target=start_websocket).start()
    now = datetime.datetime.now()
    print("Current Time:", now.strftime("%Y-%m-%d %H:%M:%S"))
    # start_websocket()
    # order_and_balance_update()

LSTM Model loaded successfully.
CNN Model loaded successfully.
Quantiles loaded successfully.
All models are ready for prediction.
Data loaded successfully. Rows: 490
Current Time: 2025-05-17 15:01:13
 * Serving Flask app '__main__'
 * Debug mode: on
Connected to CoinDCX WebSocket!


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.5:5000
Press CTRL+C to quit


Price: 217482.07, Balance: 10000.00, Position: None, Shares: 0, Next_Close: 222164.56, Close_Diff: 4682.49, Signal: 0, Order Status: filled, LSTM: 4682.4872185516, CNN: 2, CNN Prob.: 0.57611483335495     

127.0.0.1 - - [17/May/2025 15:04:12] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2025 15:04:12] "POST /update HTTP/1.1" 200 -


LSTM Model loaded successfully.
All models are ready for prediction.
Price: 217296.30, Balance: 10000.00, Position: None, Shares: 0, Next_Close: 218867.85, Close_Diff: 1571.55, Signal: 0, Order Status: filled, LSTM: 1571.552687835676, CNN: 2, CNN Prob.: 0.5761089324951172 

...

127.0.0.1 - - [17/May/2025 18:10:05] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2025 18:10:06] "POST /update HTTP/1.1" 200 -


LSTM Model loaded successfully.
All models are ready for prediction.
Price: 216815.73, Balance: 10000.00, Position: None, Shares: 0, Next_Close: 216074.45, Close_Diff: -741.28, Signal: 0, Order Status: filled, LSTM: -741.2798051643767, CNN: 2, CNN Prob.: 0.576114296913147   

127.0.0.1 - - [17/May/2025 18:19:52] "POST /update HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2025 18:19:52] "POST /update HTTP/1.1" 200 -


LSTM Model loaded successfully.
All models are ready for prediction.
Price: 216449.35, Balance: 10000.00, Position: None, Shares: 0, Next_Close: 216776.63, Close_Diff: 327.28, Signal: 0, Order Status: filled, LSTM: 327.2787326812395, CNN: 2, CNN Prob.: 0.5761109590530396   

packet queue is empty, aborting


In [ ]:
import numpy as np
import time
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import json
import requests
import datetime
from datetime import timedelta
import threading
import sys
from flask import Flask, request, jsonify
import socketio
import talib
import hmac
import hashlib
from pyts.image import GramianAngularField
from tensorflow.nn import softmax
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Global variables initialization
current_btc_price = None
cnn_model = None
lstm_model = None
order_status = None
app = Flask(__name__)
upper_quantile = None
lower_quantile = None
lstm_mae = 2000
models_loaded = False
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
live_data = []
last_processed_minute = None
current_status = ""
lookback = 10
window_size = 10
order_status = "filled"
paper_trade = None
last_price = None
current_prediction_status = ""
last_status_length = 0  # Track length of last status message

# Thread lock for trade operations
trade_lock = threading.Lock()

def place_order(order_type_buy_sell, price, shares):
    global current_btc_price, order_status
    if 1==1:
        if order_status == 'filled':
            key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
            secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
            
            secret_bytes = bytes(secret, encoding='utf-8')
            timeStamp = int(round(time.time() * 1000))
            
            # if order_type_buy_sell.lower() == "buy":
            #     price_per_unit = current_btc_price * 1.001
            # else:
            #     price_per_unit = current_btc_price * 0.999
            if order_type_buy_sell.lower() == "buy":
                price_per_unit = current_btc_price
            else:
                price_per_unit = current_btc_price
            
            body = {
                "side": order_type_buy_sell,
                "order_type": "limit_order",
                "market": "BTCINR",
                "total_quantity": 0.0002,
                "timestamp": timeStamp,
                "price_per_unit": round(price,2)
            }
            
            json_body = json.dumps(body, separators=(',', ':'))
            signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
            
            url = "https://api.coindcx.com/exchange/v1/orders/create"
            headers = {
                'Content-Type': 'application/json',
                'X-AUTH-APIKEY': key,
                'X-AUTH-SIGNATURE': signature
            }
            
            print(f"\nPlacing {order_type_buy_sell} order: {shares} BTC at {price_per_unit:.2f} (Market: {current_btc_price:.2f})")
            
            response = requests.post(url, data=json_body, headers=headers)
            if response.status_code == 200:
                try:
                    order_response = response.json()
                    print("Order response:", order_response)
                    # order_status = 'open'
                    # print(f"Order response: {order_response}")
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON: {e}")
                    print(f"Response text: {response.text}")
            else:
                print(f"Request failed with status code {response.status_code}")
                print(f"Response text: {response.text}")
        else:
            print("Order already pending")
        return None

def update_status_display(status):
    """Helper function to update status display on same line"""
    global last_status_length
    # Clear the previous status by overwriting with spaces
    sys.stdout.write('\r' + ' ' * last_status_length)
    # Write the new status
    sys.stdout.write('\r' + status)
    sys.stdout.flush()
    last_status_length = len(status)

def clean_data(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df.fillna(0, inplace=True)
    return df

def load_models(models='both'):
    global lstm_model, cnn_model, upper_quantile, lower_quantile, models_loaded
    try:
        if models == "both":
            lstm_model = load_model("lstm_model_upstox_optimized.h5", compile=False)
            cnn_model = load_model("cnn_model_upstox_optimized.h5", compile=False)
            print("Models loaded successfully.")
        elif models == "LSTM":
            lstm_model = load_model("lstm_model_upstox_optimized.h5", compile=False)
            print("LSTM Model loaded successfully.")
        elif models == "CNN":
            cnn_model = load_model("cnn_model_upstox_optimized.h5", compile=False)
            print("CNN Model loaded successfully.")
            
        if lstm_model is not None and cnn_model is not None:
            models_loaded = True
            print("All models are ready for prediction.")
        else:
            print("Warning: Some models failed to load")
            
    except Exception as e:
        print(f"Error loading models: {e}")
        models_loaded = False

def fetch_and_preprocess_data():
    end_time = int(datetime.datetime.now().timestamp() * 1000)
    start_time = int((datetime.datetime.now() - timedelta(days=5)).timestamp() * 1000)

    url = f"https://public.coindcx.com/market_data/candles?pair=I-BTC_INR&interval=1m&startTime={start_time}&endTime={end_time}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if not isinstance(data, list) or len(data) == 0:
            print("Error: API returned invalid or empty data.")
            return None, None, None

        btc_data = pd.DataFrame(data)
        btc_data = btc_data.rename(columns={
            'open': 'Open', 
            'high': 'High', 
            'low': 'Low', 
            'volume': 'Volume', 
            'close': 'Close'
        })

        btc_data['time'] = pd.to_datetime(btc_data['time'], unit='ms')
        btc_data[['Open', 'High', 'Low', 'Close', 'Volume']] = btc_data[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
        btc_data = clean_data(btc_data)

        try:
            btc_data['rsi'] = talib.RSI(btc_data['Close'], timeperiod=14)
            btc_data['macd'], btc_data['macd_signal'], btc_data['macd_hist'] = talib.MACD(
                btc_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9
            )
            btc_data['upperband'], btc_data['middleband'], btc_data['lowerband'] = talib.BBANDS(
                btc_data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
            )
            btc_data['ema_50'] = talib.EMA(btc_data['Close'], timeperiod=50)
            btc_data['ema_200'] = talib.EMA(btc_data['Close'], timeperiod=200)
            
            btc_data = clean_data(btc_data)
            btc_data['Next_Close'] = btc_data['Close'].shift(-1)
            btc_data.dropna(inplace=True)
            
            feature_cols = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 
                          'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
            
            btc_data[feature_cols] = scaler_x.fit_transform(btc_data[feature_cols])
            btc_data['Next_Close'] = scaler_y.fit_transform(btc_data[['Next_Close']])
            
            print(f"Data loaded successfully. Rows: {len(btc_data)}")
            return btc_data, scaler_x, scaler_y
            
        except Exception as e:
            print(f"Error calculating indicators: {e}")
            return None, None, None
            
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None, None, None

def preprocess_data(data):
    try:
        if data is None or not isinstance(data, pd.DataFrame):
            print("Error: Invalid input data")
            return None

        if 'Close' not in data.columns:
            print("Error: 'Close' column missing")
            return None

        data['rsi'] = talib.RSI(data['Close'], timeperiod=14)
        data['macd'], data['macd_signal'], data['macd_hist'] = talib.MACD(
            data['Close'], fastperiod=12, slowperiod=26, signalperiod=9
        )
        data['upperband'], data['middleband'], data['lowerband'] = talib.BBANDS(
            data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
        )
        data['ema_50'] = talib.EMA(data['Close'], timeperiod=50)
        data['ema_200'] = talib.EMA(data['Close'], timeperiod=200)
        
        data = clean_data(data)
        return data
        
    except Exception as e:
        print(f"Error in preprocess_data: {e}")
        return None

def calculate_dynamic_factors(close_diff):
    return 0.1, 0.5

class PaperTrade:
    def __init__(self, balance=10000):
        self.balance = balance
        self.position = None
        self.entry_price = None
        self.stop_loss = None
        self.target = None
        self.shares = 0
        self.investment_per_trade = 8000  # Fixed amount to invest per trade
        self.taker_fee = 0.0005
        self.maker_fee = 0.0002

    def execute_trade(self, signal, price, predicted_close_diff):
        global current_status, current_prediction_status, order_status
        # if signal==1:
        #     print(signal, price, predicted_close_diff)
        if 1==1:
            if order_status != 'filled':
                return

            target_pct, stop_loss_pct = calculate_dynamic_factors(abs(predicted_close_diff))
            trade_log = ""
            fee = 0
            
            if signal == 1 and self.position is None and order_status == 'filled':
                if self.balance >= self.investment_per_trade:
                    self.position = "long"
                    self.entry_price = price
                    self.stop_loss = price - abs(predicted_close_diff) * stop_loss_pct
                    self.target = price + abs(predicted_close_diff) * target_pct
                    self.shares = 0.0002
                    
                    fee = self.entry_price * self.shares * self.maker_fee
                    self.balance -= (self.entry_price * self.shares + fee)
                    # place_order("buy", price, self.shares)
                    # self.balance = fetch_balance()
                    trade_log = (f"🚀📈 LONG ENTRY: {price:.2f} | "
                               f"Target: {self.target:.2f} | "
                               f"SL: {self.stop_loss:.2f} | "
                               f"Qty: {self.shares:.6f} | "
                               f"Fee: {fee:.2f} (0.02%) | "
                               f"Bal: {self.balance:.2f}")

            elif signal == 2 and self.position is None and order_status == 'filled':
                if self.balance >= self.investment_per_trade:
                    self.position = "short"
                    self.entry_price = price
                    self.stop_loss = price + abs(predicted_close_diff) * stop_loss_pct
                    self.target = price - abs(predicted_close_diff) * target_pct
                    self.shares = self.investment_per_trade / price
                    
                    fee = self.entry_price * self.shares * self.maker_fee
                    self.balance -= fee
                    
                    trade_log = (f"📉🔻 SHORT ENTRY: {price:.2f} | "
                               f"Target: {self.target:.2f} | "
                               f"SL: {self.stop_loss:.2f} | "
                               f"Qty: {self.shares:.6f} | "
                               f"Fee: {fee:.2f} (0.02%) | "
                               f"Bal: {self.balance:.2f}")
                
            elif self.position == "long" and (price >= self.target or price <= self.stop_loss) and order_status == 'filled':
                exit_amount = price * self.shares
                pnl = (price - self.entry_price) * self.shares
                fee = exit_amount * self.maker_fee
                
                self.balance += (exit_amount - fee)
                self.position = None
                # place_order("sell", price, self.shares)
                # self.balance = fetch_balance()
                trade_log = (f"✅📈 LONG EXIT: {price:.2f} | "
                            f"PnL: {pnl:.2f} | "
                            f"Fee: {fee:.2f} (0.02%) | "
                            f"Bal: {self.balance:.2f}")

            elif self.position == "short" and (price <= self.target or price >= self.stop_loss) and order_status == 'filled':
                pnl = (self.entry_price - price) * self.shares
                fee = price * self.shares * self.maker_fee
                
                self.balance += (pnl - fee)
                self.position = None
                
                trade_log = (f"✅📉 SHORT EXIT: {price:.2f} | "
                            f"PnL: {pnl:.2f} | "
                            f"Fee: {fee:.2f} (0.02%) | "
                            f"Bal: {self.balance:.2f}")

            if trade_log:
                print(f"\n{trade_log}")
                with open("14_may_5.txt", "a") as log_file:
                    log_file.write(trade_log + "\n")

def predict(list_of_close):
    global current_status
    
    try:
        if not models_loaded or lstm_model is None or cnn_model is None:
            current_status = "Models not ready"
            return
            
        df = pd.DataFrame(list_of_close, columns=["Close"])
        df = preprocess_data(df)
        
        if df is None:
            current_status = "Preprocessing failed"
            return
            
        features = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 
                   'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
        
        missing_features = [f for f in features if f not in df.columns]
        if missing_features:
            current_status = f"Missing features: {missing_features}"
            return
            
        scaled_data = scaler_x.transform(df[features])
        
        if np.isnan(scaled_data).any() or np.isinf(scaled_data).any():
            current_status = "Scaled data contains NaN/inf"
            return
            
        if scaled_data.shape[0] < lookback:
            return
    
        lstm_X = np.array([scaled_data[-lookback:]]).reshape((1, lookback, len(features)))
        predicted_next_close_scaled = lstm_model.predict(lstm_X, verbose=0)[0][0]
        predicted_next_close = scaler_y.inverse_transform([[predicted_next_close_scaled]])[0][0]
        
        gaf = GramianAngularField(method="summation")
        gaf_image = gaf.fit_transform(scaled_data[-window_size:].T)
        cnn_X = np.expand_dims(gaf_image, axis=0)
        cnn_X = np.transpose(cnn_X, (0, 2, 3, 1))
        
        cnn_pred = cnn_model.predict(cnn_X, verbose=0)
        class_probabilities = softmax(cnn_pred).numpy()[0]
        predicted_class = np.argmax(cnn_pred, axis=1)[0]
        predicted_class_prob = class_probabilities[predicted_class]
    
        close_diff = predicted_next_close - list_of_close[-1]
        signal = 0
        
        if close_diff > 0 and predicted_class == 1:
            signal = 1
        # if close_diff > 0:
        #     signal = 1
            
        paper_trade.execute_trade(signal, list_of_close[-1], close_diff)
        
        current_status = (
            f"Price: {list_of_close[-1]:.2f}, Balance: {paper_trade.balance:.2f}, "
            f"Position: {paper_trade.position if paper_trade.position else 'None'}, "
            f"Shares: {paper_trade.shares:.6f}, Next_Close: {predicted_next_close:.2f}, "
            f"Close_Diff: {close_diff:.2f}, Signal: {signal}, "
            f"Order Status: {order_status}, LSTM: {close_diff}, "
            f"CNN: {predicted_class}, CNN Prob.: {predicted_class_prob:.4f}"
        )
        
        update_status_display(current_status)
    except Exception as e:
        current_status = f"Error in predict: {e}"
        update_status_display(current_status)

def order_and_balance_update():
    global order_status
    
    socketEndpoint = 'wss://stream.coindcx.com'
    
    key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
    secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
    
    secret_bytes = bytes(secret, encoding='utf-8')
    channelName = "coindcx"
    body = {"channel": channelName}
    json_body = json.dumps(body, separators=(',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    sio2 = socketio.Client()

    @sio2.event
    def connect():
        print("Connected to order channel!")
        sio2.emit('join', {'channelName': channelName, 'authSignature': signature, 'apiKey': key})
    
    @sio2.on('order-update')
    def on_message1(response):
        global order_status
        if isinstance(response, dict) and 'data' in response:
            try:
                order_data = json.loads(response['data'])
                if isinstance(order_data, list) and len(order_data) > 0:
                    status = order_data[0].get('status', 'Unknown')
                    order_status = status
            except json.JSONDecodeError:
                pass

    sio2.connect(socketEndpoint, transports=['websocket'])
    sio2.wait()

def start_websocket():
    socketEndpoint = 'wss://stream.coindcx.com'
    sio = socketio.Client()

    @sio.event
    def connect():
        print("Connected to CoinDCX WebSocket!")
        sio.emit('join', {'channelName': "I-BTC_INR@prices"})

    @sio.on('price-change')
    def on_message(message):
        global last_processed_minute, live_data, last_price, current_btc_price
        
        try:
            btc_inr_price = float(json.loads(message['data'])['p'])
            last_price = btc_inr_price
            current_btc_price = btc_inr_price
            
            current_time = datetime.datetime.now()
            current_minute = current_time.minute
            
            if last_processed_minute is None:
                last_processed_minute = current_minute

            if current_minute != last_processed_minute:
                last_processed_minute = current_minute

                if len(live_data) >= lookback:
                    live_data.pop(0)

                live_data.append(btc_inr_price)
                data_to_pass = live_data.copy()
            else:
                data_to_pass = live_data[-(lookback-1):] + [btc_inr_price]
                
            predict(data_to_pass)
            
        except Exception as e:
            current_status = f"Error in websocket: {e}"
            update_status_display(current_status)

    sio.connect(socketEndpoint, transports=['websocket'])
    sio.wait()

@app.route('/update', methods=['POST'])
def update():
    global lstm_mae
    data = request.get_json()
    if 'Updated' in data:
        load_models(models=data['Updated'])
        return jsonify({'message': 'Update received successfully'}), 200
    if 'LSTM' in data:
        lstm_mae = int(data['LSTM'])
        return jsonify({'message': 'LSTM mae received successfully'}), 200
    return jsonify({'error': 'Missing parameter'}), 400

def fetch_balance():
    key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
    secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
    
    secret_bytes = bytes(secret, encoding='utf-8')
    timeStamp = int(round(time.time() * 1000))
    
    body = {"timestamp": timeStamp}
    json_body = json.dumps(body, separators=(',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    url = "https://api.coindcx.com/exchange/v1/users/balances"
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }
    
    response = requests.post(url, data=json_body, headers=headers)
    data = response.json()
    btc_balance = next((item['balance'] for item in data if item['currency'] == 'INR'), None)
    return btc_balance

if __name__ == '__main__':
    load_models(models='both')
    btc_data, scaler_x, scaler_y = fetch_and_preprocess_data()

    if btc_data is None or scaler_x is None or scaler_y is None:
        print("Failed to initialize data. Exiting...")
        sys.exit(1)

    live_data = btc_data['Close'].tolist()[-lookback:]
    paper_trade = PaperTrade(balance=10000)

    threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)).start()
    # threading.Thread(target=order_and_balance_update).start()
    now = datetime.datetime.now()
    print("Current Time:", now.strftime("%Y-%m-%d %H:%M:%S"))
    start_websocket()

In [ ]:
import numpy as np
import time
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import json
import requests
import datetime
from datetime import timedelta
import threading
import sys
from flask import Flask, request, jsonify
import socketio
import talib
import hmac
import hashlib
from pyts.image import GramianAngularField
from tensorflow.nn import softmax
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Global variables initialization
current_btc_price = None
cnn_model = None
lstm_model = None
order_status = None
app = Flask(__name__)
upper_quantile = None
lower_quantile = None
lstm_mae = 2000
models_loaded = False
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
live_data = []
last_processed_minute = None
current_status = ""
lookback = 10
window_size = 10
order_status = "filled"
paper_trade = None
last_price = None
current_prediction_status = ""
last_status_length = 0  # Track length of last status message

# Thread lock for trade operations
trade_lock = threading.Lock()

def place_order(order_type_buy_sell, price, shares):
    global current_btc_price, order_status
    if 1==1:
        if order_status == 'filled':
            key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
            secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
            
            secret_bytes = bytes(secret, encoding='utf-8')
            timeStamp = int(round(time.time() * 1000))
            
            if order_type_buy_sell.lower() == "buy":
                price_per_unit = current_btc_price
            else:
                price_per_unit = current_btc_price
            
            body = {
                "side": order_type_buy_sell,
                "order_type": "limit_order",
                "market": "BTCINR",
                "total_quantity": 0.0002,
                "timestamp": timeStamp,
                "price_per_unit": round(price,2)
            }
            
            json_body = json.dumps(body, separators=(',', ':'))
            signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
            
            url = "https://api.coindcx.com/exchange/v1/orders/create"
            headers = {
                'Content-Type': 'application/json',
                'X-AUTH-APIKEY': key,
                'X-AUTH-SIGNATURE': signature
            }
            
            print(f"\nPlacing {order_type_buy_sell} order: {shares} BTC at {price_per_unit:.2f} (Market: {current_btc_price:.2f})")
            
            response = requests.post(url, data=json_body, headers=headers)
            if response.status_code == 200:
                try:
                    order_response = response.json()
                    print("Order response:", order_response)
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON: {e}")
                    print(f"Response text: {response.text}")
            else:
                print(f"Request failed with status code {response.status_code}")
                print(f"Response text: {response.text}")
        else:
            print("Order already pending")
        return None

def update_status_display(status):
    """Helper function to update status display on same line"""
    global last_status_length
    sys.stdout.write('\r' + ' ' * last_status_length)
    sys.stdout.write('\r' + status)
    sys.stdout.flush()
    last_status_length = len(status)

def clean_data(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    df.fillna(0, inplace=True)
    return df

def load_models(models='both'):
    global lstm_model, cnn_model, upper_quantile, lower_quantile, models_loaded
    try:
        if models == "both":
            lstm_model = load_model("lstm_model_upstox_optimized.h5", compile=False)
            cnn_model = load_model("cnn_model_upstox_optimized.h5", compile=False)
            print("Models loaded successfully.")
            if cnn_model:
                print("CNN Model Summary:")
                cnn_model.summary()
        elif models == "LSTM":
            lstm_model = load_model("lstm_model_upstox_optimized.h5", compile=False)
            print("LSTM Model loaded successfully.")
        elif models == "CNN":
            cnn_model = load_model("cnn_model_upstox_optimized.h5", compile=False)
            print("CNN Model loaded successfully.")
            if cnn_model:
                print("CNN Model Summary:")
                cnn_model.summary()
            
        if lstm_model is not None and cnn_model is not None:
            models_loaded = True
            print("All models are ready for prediction.")
        else:
            print("Warning: Some models failed to load")
            
    except Exception as e:
        print(f"Error loading models: {e}")
        models_loaded = False

def fetch_and_preprocess_data():
    end_time = int(datetime.datetime.now().timestamp() * 1000)
    start_time = int((datetime.datetime.now() - timedelta(days=5)).timestamp() * 1000)

    url = f"https://public.coindcx.com/market_data/candles?pair=I-BTC_INR&interval=1m&startTime={start_time}&endTime={end_time}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if not isinstance(data, list) or len(data) == 0:
            print("Error: API returned invalid or empty data.")
            return None, None, None

        btc_data = pd.DataFrame(data)
        btc_data = btc_data.rename(columns={
            'open': 'Open', 
            'high': 'High', 
            'low': 'Low', 
            'volume': 'Volume', 
            'close': 'Close'
        })

        btc_data['time'] = pd.to_datetime(btc_data['time'], unit='ms')
        btc_data[['Open', 'High', 'Low', 'Close', 'Volume']] = btc_data[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
        btc_data = clean_data(btc_data)

        try:
            btc_data['rsi'] = talib.RSI(btc_data['Close'], timeperiod=14)
            btc_data['macd'], btc_data['macd_signal'], btc_data['macd_hist'] = talib.MACD(
                btc_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9
            )
            btc_data['upperband'], btc_data['middleband'], btc_data['lowerband'] = talib.BBANDS(
                btc_data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
            )
            btc_data['ema_50'] = talib.EMA(btc_data['Close'], timeperiod=50)
            btc_data['ema_200'] = talib.EMA(btc_data['Close'], timeperiod=200)
            
            btc_data = clean_data(btc_data)
            btc_data['Next_Close'] = btc_data['Close'].shift(-1)
            btc_data.dropna(inplace=True)
            
            feature_cols = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 
                          'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
            
            btc_data[feature_cols] = scaler_x.fit_transform(btc_data[feature_cols])
            btc_data['Next_Close'] = scaler_y.fit_transform(btc_data[['Next_Close']])
            
            print(f"Data loaded successfully. Rows: {len(btc_data)}")
            return btc_data, scaler_x, scaler_y
            
        except Exception as e:
            print(f"Error calculating indicators: {e}")
            return None, None, None
            
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None, None, None

def preprocess_data(data):
    try:
        if data is None or not isinstance(data, pd.DataFrame):
            print("Error: Invalid input data")
            return None

        if 'Close' not in data.columns:
            print("Error: 'Close' column missing")
            return None

        data['rsi'] = talib.RSI(data['Close'], timeperiod=14)
        data['macd'], data['macd_signal'], data['macd_hist'] = talib.MACD(
            data['Close'], fastperiod=12, slowperiod=26, signalperiod=9
        )
        data['upperband'], data['middleband'], data['lowerband'] = talib.BBANDS(
            data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
        )
        data['ema_50'] = talib.EMA(data['Close'], timeperiod=50)
        data['ema_200'] = talib.EMA(data['Close'], timeperiod=200)
        
        data = clean_data(data)
        return data
        
    except Exception as e:
        print(f"Error in preprocess_data: {e}")
        return None

def calculate_dynamic_factors(close_diff):
    return 0.1, 0.5

class PaperTrade:
    def __init__(self, balance=10000):
        self.balance = balance
        self.position = None
        self.entry_price = None
        self.stop_loss = None
        self.target = None
        self.shares = 0
        self.investment_per_trade = 8000  # Fixed amount to invest per trade
        self.taker_fee = 0.0005
        self.maker_fee = 0.0002

    def execute_trade(self, signal, price, predicted_close_diff):
        global current_status, current_prediction_status, order_status
        if 1==1:
            if order_status != 'filled':
                return

            target_pct, stop_loss_pct = calculate_dynamic_factors(abs(predicted_close_diff))
            trade_log = ""
            fee = 0
            
            if signal == 1 and self.position is None and order_status == 'filled':
                if self.balance >= self.investment_per_trade:
                    self.position = "long"
                    self.entry_price = price
                    self.stop_loss = price - abs(predicted_close_diff) * stop_loss_pct
                    self.target = price + abs(predicted_close_diff) * target_pct
                    self.shares = 0.0002
                    
                    fee = self.entry_price * self.shares * self.maker_fee
                    self.balance -= (self.entry_price * self.shares + fee)
                    trade_log = (f"🚀📈 LONG ENTRY: {price:.2f} | "
                               f"Target: {self.target:.2f} | "
                               f"SL: {self.stop_loss:.2f} | "
                               f"Qty: {self.shares:.6f} | "
                               f"Fee: {fee:.2f} (0.02%) | "
                               f"Bal: {self.balance:.2f}")

            elif signal == 2 and self.position is None and order_status == 'filled':
                if self.balance >= self.investment_per_trade:
                    self.position = "short"
                    self.entry_price = price
                    self.stop_loss = price + abs(predicted_close_diff) * stop_loss_pct
                    self.target = price - abs(predicted_close_diff) * target_pct
                    self.shares = self.investment_per_trade / price
                    
                    fee = self.entry_price * self.shares * self.maker_fee
                    self.balance -= fee
                    
                    trade_log = (f"📉🔻 SHORT ENTRY: {price:.2f} | "
                               f"Target: {self.target:.2f} | "
                               f"SL: {self.stop_loss:.2f} | "
                               f"Qty: {self.shares:.6f} | "
                               f"Fee: {fee:.2f} (0.02%) | "
                               f"Bal: {self.balance:.2f}")
                
            elif self.position == "long" and (price >= self.target or price <= self.stop_loss) and order_status == 'filled':
                exit_amount = price * self.shares
                pnl = (price - self.entry_price) * self.shares
                fee = exit_amount * self.maker_fee
                
                self.balance += (exit_amount - fee)
                self.position = None
                trade_log = (f"✅📈 LONG EXIT: {price:.2f} | "
                            f"PnL: {pnl:.2f} | "
                            f"Fee: {fee:.2f} (0.02%) | "
                            f"Bal: {self.balance:.2f}")

            elif self.position == "short" and (price <= self.target or price >= self.stop_loss) and order_status == 'filled':
                pnl = (self.entry_price - price) * self.shares
                fee = price * self.shares * self.maker_fee
                
                self.balance += (pnl - fee)
                self.position = None
                
                trade_log = (f"✅📉 SHORT EXIT: {price:.2f} | "
                            f"PnL: {pnl:.2f} | "
                            f"Fee: {fee:.2f} (0.02%) | "
                            f"Bal: {self.balance:.2f}")

            if trade_log:
                print(f"\n{trade_log}")
                with open("14_may_5.txt", "a") as log_file:
                    log_file.write(trade_log + "\n")

def predict(list_of_close):
    global current_status
    
    try:
        if not models_loaded or lstm_model is None or cnn_model is None:
            current_status = "Models not ready"
            return
            
        df = pd.DataFrame(list_of_close, columns=["Close"])
        df = preprocess_data(df)
        
        if df is None:
            current_status = "Preprocessing failed"
            return
            
        features = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 
                   'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
        
        missing_features = [f for f in features if f not in df.columns]
        if missing_features:
            current_status = f"Missing features: {missing_features}"
            return
            
        scaled_data = scaler_x.transform(df[features])
        
        if np.isnan(scaled_data).any() or np.isinf(scaled_data).any():
            current_status = "Scaled data contains NaN/inf"
            return
            
        if scaled_data.shape[0] < lookback:
            return
    
        # LSTM Prediction
        lstm_X = np.array([scaled_data[-lookback:]]).reshape((1, lookback, len(features)))
        predicted_next_close_scaled = lstm_model.predict(lstm_X, verbose=0)[0][0]
        predicted_next_close = scaler_y.inverse_transform([[predicted_next_close_scaled]])[0][0]
        close_diff = predicted_next_close - list_of_close[-1]
        
        # CNN Prediction (GAF Transformation)
        gaf = GramianAngularField(method="summation")
        gaf_data = scaled_data[-window_size:]
        
        # Ensure correct shape for GAF (n_samples, n_features)
        if gaf_data.shape[0] < window_size:
            current_status = "Not enough data for GAF"
            return
            
        gaf_image = gaf.fit_transform(gaf_data.T)  # Transpose for correct GAF input
        
        # Debug GAF output shape
        print(f"GAF Image Shape: {gaf_image.shape}")  # Should be (n_features, window_size, window_size)
        
        # Reshape for CNN input (add batch and channel dim)
        cnn_X = np.expand_dims(gaf_image, axis=0)  # Add batch dimension
        cnn_X = np.transpose(cnn_X, (0, 2, 3, 1))    # Shape: (1, window_size, window_size, n_features)
        
        print(f"CNN Input Shape: {cnn_X.shape}")  # Debug shape
        
        # Get CNN prediction
        cnn_pred = cnn_model.predict(cnn_X, verbose=0)
        print(f"Raw CNN Output: {cnn_pred}")  # Debug raw prediction
        
        # If CNN is classifier (buy/sell/hold)
        predicted_class = np.argmax(cnn_pred, axis=1)[0]
        predicted_class_prob = np.max(softmax(cnn_pred).numpy()[0])
        
        # Determine signal
        signal = 0
        if close_diff > 0 and predicted_class == 1:  # Assuming class 1 = buy
            signal = 1
        elif close_diff < 0 and predicted_class == 2:  # Assuming class 2 = sell
            signal = 2
        
        paper_trade.execute_trade(signal, list_of_close[-1], close_diff)
        
        current_status = (
            f"Price: {list_of_close[-1]:.2f}, Balance: {paper_trade.balance:.2f}, "
            f"Position: {paper_trade.position if paper_trade.position else 'None'}, "
            f"Shares: {paper_trade.shares:.6f}, Next_Close: {predicted_next_close:.2f}, "
            f"Close_Diff: {close_diff:.2f}, Signal: {signal}, "
            f"Order Status: {order_status}, LSTM: {close_diff:.2f}, "
            f"CNN Class: {predicted_class}, CNN Prob: {predicted_class_prob:.4f}"
        )
        
        update_status_display(current_status)
    except Exception as e:
        current_status = f"Error in predict: {str(e)}"
        update_status_display(current_status)

def order_and_balance_update():
    global order_status
    
    socketEndpoint = 'wss://stream.coindcx.com'
    
    key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
    secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
    
    secret_bytes = bytes(secret, encoding='utf-8')
    channelName = "coindcx"
    body = {"channel": channelName}
    json_body = json.dumps(body, separators=(',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    sio2 = socketio.Client()

    @sio2.event
    def connect():
        print("Connected to order channel!")
        sio2.emit('join', {'channelName': channelName, 'authSignature': signature, 'apiKey': key})
    
    @sio2.on('order-update')
    def on_message1(response):
        global order_status
        if isinstance(response, dict) and 'data' in response:
            try:
                order_data = json.loads(response['data'])
                if isinstance(order_data, list) and len(order_data) > 0:
                    status = order_data[0].get('status', 'Unknown')
                    order_status = status
            except json.JSONDecodeError:
                pass

    sio2.connect(socketEndpoint, transports=['websocket'])
    sio2.wait()

def start_websocket():
    socketEndpoint = 'wss://stream.coindcx.com'
    sio = socketio.Client()

    @sio.event
    def connect():
        print("Connected to CoinDCX WebSocket!")
        sio.emit('join', {'channelName': "I-BTC_INR@prices"})

    @sio.on('price-change')
    def on_message(message):
        global last_processed_minute, live_data, last_price, current_btc_price
        
        try:
            btc_inr_price = float(json.loads(message['data'])['p'])
            last_price = btc_inr_price
            current_btc_price = btc_inr_price
            
            current_time = datetime.datetime.now()
            current_minute = current_time.minute
            
            if last_processed_minute is None:
                last_processed_minute = current_minute

            if current_minute != last_processed_minute:
                last_processed_minute = current_minute

                if len(live_data) >= lookback:
                    live_data.pop(0)

                live_data.append(btc_inr_price)
                data_to_pass = live_data.copy()
            else:
                data_to_pass = live_data[-(lookback-1):] + [btc_inr_price]
                
            predict(data_to_pass)
            
        except Exception as e:
            current_status = f"Error in websocket: {e}"
            update_status_display(current_status)

    sio.connect(socketEndpoint, transports=['websocket'])
    sio.wait()

@app.route('/update', methods=['POST'])
def update():
    global lstm_mae
    data = request.get_json()
    if 'Updated' in data:
        load_models(models=data['Updated'])
        return jsonify({'message': 'Update received successfully'}), 200
    if 'LSTM' in data:
        lstm_mae = int(data['LSTM'])
        return jsonify({'message': 'LSTM mae received successfully'}), 200
    return jsonify({'error': 'Missing parameter'}), 400

def fetch_balance():
    key = "25cfd2c369224d983d263096beb1748faf683f54d1468435"
    secret = "a0501522facc1a37495020de16053ddcbb5906c88204fdda8eb0b140e09529ab"
    
    secret_bytes = bytes(secret, encoding='utf-8')
    timeStamp = int(round(time.time() * 1000))
    
    body = {"timestamp": timeStamp}
    json_body = json.dumps(body, separators=(',', ':'))
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()
    
    url = "https://api.coindcx.com/exchange/v1/users/balances"
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }
    
    response = requests.post(url, data=json_body, headers=headers)
    data = response.json()
    btc_balance = next((item['balance'] for item in data if item['currency'] == 'INR'), None)
    return btc_balance

if __name__ == '__main__':
    load_models(models='both')
    btc_data, scaler_x, scaler_y = fetch_and_preprocess_data()

    if btc_data is None or scaler_x is None or scaler_y is None:
        print("Failed to initialize data. Exiting...")
        sys.exit(1)

    live_data = btc_data['Close'].tolist()[-lookback:]
    paper_trade = PaperTrade(balance=10000)

    threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)).start()
    threading.Thread(target=start_websocket).start()
    now = datetime.datetime.now()
    print("Current Time:", now.strftime("%Y-%m-%d %H:%M:%S"))
    